In [1]:
# ## Importieren der notwendigen Bibliotheken
import chess.pgn
import chess.svg
import random as ran
import numpy as np
from tensorflow.python.keras.utils.version_utils import callbacks
import tensorflow as tf
# ## Laden der PGN-Datei
pgn = open("data/lichess_db_standard_rated_2013-01.pgn", encoding="utf-8-sig")

In [2]:
# ## Initialisieren der Liste für Schachmatt-Spiele
checkmate_games = []

# ## Extrahieren der Spiele, die durch Schachmatt beendet wurden
while True:
    offset = pgn.tell()
    headers = chess.pgn.read_headers(pgn)

    if headers is None:
        break
    if "Normal" in headers.get("Termination"):
        checkmate_games.append(offset)
print(f"Anzahl der Spiele: {len(checkmate_games)}")        

Anzahl der Spiele: 83449


In [3]:
# ## Laden der Schachmatt-Spiele in eine Liste
checkmate_games2 = []
for i in checkmate_games:
    pgn.seek(i)
    checkmate_games2.append(chess.pgn.read_game(pgn))
print(f"Anzahl der Spiele, die unentschieden oder in Schachmatt enden: {len(checkmate_games2)}")

Anzahl der Spiele, die unentschieden oder in Schachmatt enden: 83449


In [4]:
# ## Filtern der tatsächlichen Schachmatt-Spiele
checkmate_games3 = []

for i in range(len(checkmate_games2)):
    if checkmate_games2[i].end().board().is_checkmate():
        checkmate_games3.append(checkmate_games2[i])

print(f"Anzahl der Schachmatt-Spiele: {len(checkmate_games3)}")


Anzahl der Schachmatt-Spiele: 34475


In [11]:
# ## Umwandeln der Schachmatt-Endstellungen in Listenform für Eingabe in ANN 
checkmate_as_list = []
for i in range(len(checkmate_games3)):
    board = checkmate_games3[i].end().board()

    l = [None] * 64
    for sq in chess.scan_reversed(board.occupied_co[chess.WHITE]):  # Check if white
        l[sq] = board.piece_type_at(sq)
    for sq in chess.scan_reversed(board.occupied_co[chess.BLACK]):  # Check if black
        l[sq] = -board.piece_type_at(sq)
    checkmate_as_list.append([0 if v is None else v for v in l])
print(checkmate_as_list[0])

[4, 0, 0, 0, 6, 0, 0, 4, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, -5, 0, 0, 0, 0, 0, 0, 0, 3, 0, -1, -2, 0, -1, 0, 0, 0, -1, -3, -1, -1, 0, 0, 0, -1, -4, 0, 0, -6, 5, -3, 0, -4]


In [6]:
# ## Erstellen von Nicht-Schachmatt-Stellungen
non_checkmate_as_list = []
for j in range(1, 70000):
    board = checkmate_games2[j].board()
    moves = []

    for move in checkmate_games2[j].mainline_moves():
        moves.append(move)

    if len(moves) - 2 <= 1:
        continue

    r1 = ran.randint(1, len(moves) - 2)

    for i in range(r1):
        board.push(moves[i])

    l = [None] * 64
    for sq in chess.scan_reversed(board.occupied_co[chess.WHITE]):  # Check if white
        l[sq] = board.piece_type_at(sq)
    for sq in chess.scan_reversed(board.occupied_co[chess.BLACK]):  # Check if black
        l[sq] = -board.piece_type_at(sq)
    non_checkmate_as_list.append([0 if v is None else v for v in l])

In [12]:
# ## Importieren der notwendigen Bibliothek
from sklearn.model_selection import train_test_split

# ## Entfernen der Duplikate 
unique_array = np.unique(checkmate_as_list, axis=0)
mate_Imgs_ann = unique_array

unique_array = np.unique(non_checkmate_as_list, axis=0)
not_mate_Imgs_ann = unique_array

# ## Aufteilen der Daten in Trainings- und Testdaten
train1 = mate_Imgs_ann[:30000]  # 10000 checkmate images
train2 = not_mate_Imgs_ann[:30000]  # 50000 non-checkmate images

test_mate_ann = mate_Imgs_ann[30000:31000]  # 100 checkmate images
test_not_mate_ann= not_mate_Imgs_ann[30000:31000]  # 500 non-checkmate images

train1_labs = np.ones(30000)  # label checkmate images
train2_labs = np.zeros(30000)  # label non-checkmate images

# ## Zusammenführen der Trainingsdaten
training_data_features_ann = np.concatenate((train1, train2))
training_data_labels_ann = np.concatenate((train1_labs, train2_labs))


train_data_ann, test_data_ann, train_labels_ann, test_labels_ann = train_test_split(training_data_features_ann,training_data_labels_ann , test_size=0.2, random_state=42)
print(train_data_ann.shape)
print(test_data_ann.shape)


(48000, 64)


In [ ]:
patience= 10

model = tf.keras.Sequential([
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Input(shape=(64)),
    tf.keras.layers.Dense(32, activation='sigmoid'),
    tf.keras.layers.Dense(2, activation='softmax')

])

# opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                        mode="min", patience=10,
                                        restore_best_weights=True, verbose=1)
# Train the model
history = model.fit(x=train_data_ann,
                    y=train_labels_ann,
                    epochs=1000,
                    verbose=1,
                    batch_size=32,
                    validation_data=(test_data_ann, test_labels_ann),
                    callbacks=[earlystopping])

bestEpoch = earlystopping.stopped_epoch - patience

model.save('./data/models/ann_model_max_epochs.h5')

mate_pred = model.predict(test_mate_ann)
test_not_mate_pred = model.predict(test_not_mate_ann)
print(mate_pred)
print(test_not_mate_pred)

mates = [np.round(sub_array[0]) for sub_array in mate_pred]
not_mates = [np.round(sub_array[0]) for sub_array in test_not_mate_pred]

print(mates)
print(not_mates)

print(f'{mates.count(0)} of 1000 mates predicted correctly')
print(f'{not_mates.count(1)} of 1000 non-mates predicted correctly')

Epoch 1/1000
 756/1500 [==============>...............] - ETA: 2s - loss: 0.5912 - acc: 0.6878